In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
human_data = pd.read_table('../Datasets/human_dataset.txt')
labels = human_data['class'].to_numpy()
dna_spectral_representation = np.load('../DNA Vector Representations/human_dataset_dna_vector.npy')
print(dna_spectral_representation.shape)
print(labels.shape)

(4380, 1024)
(4380,)


In [3]:
X_train, X_test, Y_train, Y_test = train_test_split(dna_spectral_representation, labels, test_size = 0.2)